In [158]:
import pandas as pd
from IPython.display import HTML
import math

## Reading all CSV files with Pandas

In [3]:
Trip = pd.read_csv("trip.csv")
Trip.head()

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,4576,63,8/29/2013 14:13,South Van Ness at Market,66,8/29/2013 14:14,South Van Ness at Market,66,520,Subscriber,94127
1,4607,70,8/29/2013 14:42,San Jose City Hall,10,8/29/2013 14:43,San Jose City Hall,10,661,Subscriber,95138
2,4130,71,8/29/2013 10:16,Mountain View City Hall,27,8/29/2013 10:17,Mountain View City Hall,27,48,Subscriber,97214
3,4251,77,8/29/2013 11:29,San Jose City Hall,10,8/29/2013 11:30,San Jose City Hall,10,26,Subscriber,95060
4,4299,83,8/29/2013 12:02,South Van Ness at Market,66,8/29/2013 12:04,Market at 10th,67,319,Subscriber,94103


In [4]:
Weather = pd.read_csv("weather.csv")
Weather.head()

,Unnamed: 0,date,mean_temperature_f,mean_humidity,mean_dew_point_f,mean_wind_speed_mph,zip_code
0,0,8/29/2013,68.0,75.0,58.0,11.0,94107
1,1,8/30/2013,69.0,70.0,58.0,13.0,94107
2,2,8/31/2013,64.0,75.0,56.0,15.0,94107
3,3,9/1/2013,66.0,68.0,56.0,13.0,94107
4,4,9/2/2013,69.0,77.0,60.0,12.0,94107


In [5]:
Station = pd.read_csv("station.csv")
Station.head()

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013


In [6]:
Status = pd.read_csv("status.csv")
Status.head()

,station_id,bikes_available,docks_available,time
0,2,2,25,2013/08/29 12:06:01
1,2,2,25,2013/08/29 12:07:01
2,2,2,25,2013/08/29 12:08:01
3,2,2,25,2013/08/29 12:09:01
4,2,2,25,2013/08/29 12:10:01


## Creating DataBase & Tables

In [207]:
import sqlite3

In [206]:
conn = sqlite3.connect("Yulu.db")
cur = conn.cursor()

In [9]:
# Create Trip table
Trip.to_sql("Trip", conn, if_exists="replace", index=False)

In [10]:
# Create Weather table
Weather.to_sql("Weather", conn, if_exists="replace", index=False)

C:\Users\ark14\anaconda3\lib\site-packages\pandas\core\generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [11]:
# Create Station table
Station.to_sql("Station", conn, if_exists="replace", index=False)

In [12]:
# Create Status table
Status.to_sql("Status", conn, if_exists="replace", index=False)

# Task 1: Get to Know Your Company

## Q1.What are the total numbers of:

### Q1.1 Bike stations?

In [82]:
Total_stations = pd.read_sql_query("SELECT COUNT(id) as total_stations FROM Station",conn)
HTML(Total_stations.to_html(index=False))

total_stations
70


### Q1.2. Bikes?

In [83]:
Total_bikes = pd.read_sql_query("SELECT COUNT(distinct(bike_id)) as total_bikes FROM Trip",conn)
HTML(Total_bikes.to_html(index=False))

,total_bikes
0,700


total_bikes
700


### Q1.3 Trips?

In [85]:
Total_trips = pd.read_sql_query("SELECT COUNT(*) as total_bikes FROM Trip",conn)
HTML(Total_trips.to_html(index=False))

total_bikes
669959


### Question 1.4

### Please refer Tableau file shared

## Question 3: What is the relationship between the following columns (one to one, many to one, many to many)?



### Question 3.1 bike_id (Trip table) and start_station_id (Trip table)
### Answer: Many to many relationship i.e as seen in the below results multiple bike_id's are associated to multiple start_station_id

In [93]:
Totalstart_station_id = pd.read_sql_query("SELECT bike_id ,count(DISTINCT(start_station_id)) as Totalstart_station_id FROM Trip group by bike_id order by Totalstart_station_id limit 3",conn)
HTML(Totalstart_station_id.to_html(index=False))

bike_id,Totalstart_station_id
876,5
323,10
697,10


In [94]:
Totalbikeid = pd.read_sql_query("SELECT start_station_id ,COUNT(DISTINCT(bike_id)) as Totalbikeid FROM Trip group by start_station_id order by Totalbikeid limit 3",conn)
HTML(Totalbikeid.to_html(index=False))

start_station_id,Totalbikeid
24,109
23,116
21,119


### Question 3.2 pincode (Weather table) and station location (latitude and longitude in Station table)

### Answer: There is no Relationship between Weather table & (latitude and longitude in Station table) since there are no common columns between them (No foreign key relationship)

### Question 3.3 8/29/2013 (date column in Weather table) and mean wind speed (Weather table)

### Answer: 8/29/2013 in date column has 5 mean wind speed (as seen below), hence one date (8/29/2013) has many mean wind speed resulting in one to many relationship. But note that even one mean speed has many dates, hence overall the relationship between Date column and mean wind column is many to many but if we pick one specific date such as (8/29/2013) this is one to many (mean wind speed) relationship

In [118]:
mean_speed = pd.read_sql_query("SELECT date ,mean_wind_speed_mph as meanspeed FROM Weather where date = '8/29/2013'",conn)
HTML(mean_speed.to_html(index=False))

date,meanspeed
8/29/2013,11.0
8/29/2013,6.0
8/29/2013,8.0
8/29/2013,5.0
8/29/2013,7.0


## Question 4: Find the first and the last trip in the data

In [96]:
#First Trip
first_trip = pd.read_sql_query("SELECT * FROM Trip order by id limit 1",conn)
HTML(first_trip.to_html(index=False))

id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
4069,174,8/29/2013 9:08,2nd at South Park,64,8/29/2013 9:11,2nd at South Park,64,288,Subscriber,94114


In [97]:
#last Trip
last_trip = pd.read_sql_query("SELECT * FROM Trip order by id desc limit 1",conn)
HTML(last_trip.to_html(index=False))

id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
913460,765,8/31/2015 23:26,Harry Bridges Plaza (Ferry Building),50,8/31/2015 23:39,San Francisco Caltrain (Townsend at 4th),70,288,Subscriber,2139


## Question 5: What is the average duration

## Of all the trips?

In [131]:
# Average Duration of all trips
Average_DurationAlltrip = pd.read_sql_query("SELECT round(avg(duration)) as Average_Duration_of_allTrips FROM Trip",conn)
HTML(Average_DurationAlltrip.to_html(index=False))

Average_Duration_of_allTrips
1108.0


## Of trips on which customers are ending their rides at the same station from where they started?

In [134]:
# Average Duration of all trips with same start and end location
AvgDuration_Location = pd.read_sql_query("SELECT round(avg(duration)) as AvgDuration_sametoFroLocation FROM Trip where start_station_name = end_station_name",conn)
HTML(AvgDuration_Location.to_html(index=False))

AvgDuration_sametoFroLocation
6357.0


## Question 6: Which bike has been used the most in terms of duration? (Answer with the Bike ID)

In [103]:
# Will find sum of duration for each bike (group by bike_id) and then order by sum duration desc
most_usedBike = pd.read_sql_query("SELECT bike_id ,sum(duration) as Total_duration FROM Trip group by bike_id order by Total_duration desc limit 1",conn)
HTML(most_usedBike.to_html(index=False))

bike_id,Total_duration
535,18611693


## Question 7.1 ,7.2 & 7.3

### Please refer Tableau file shared

# Task 2: Demand Prediction

### Zulip is running under a loss and has decided to shut operations for three of its stations. You have to use the data provided to help Zulip decide which three stations should be shut.

## Question-1: What are the top 10 least popular stations? Hint: Find the least frequently appearing start stations from the Trip table

In [212]:
Ten_leastpopular = pd.read_sql_query("SELECT start_station_name ,Count(start_station_name) as leastUsed_start_station_name from Trip group by start_station_name order by leastUsed_start_station_name asc limit 10 ",conn)
HTML(Ten_leastpopular.to_html(index=False))

start_station_name,leastUsed_start_station_name
San Jose Government Center,23
Broadway at Main,67
Redwood City Public Library,213
Franklin at Maple,224
San Mateo County Center,287
Redwood City Medical Center,311
Mezes Park,341
Stanford in Redwood City,436
Park at Olive,750
Santa Clara County Civic Center,840


## Questin-2: Idle time is the duration for which a station remains inactive. You can consider this as the time for which a station has more than 3 bikes available.

### Find the idle time for Station 2 on 29 July, 2018

In [183]:
# Below shows the top 10 idle time for station 2
idleTime_Station2 = pd.read_sql_query("SELECT time as idle_time , bikes_available from Status where station_id = 2 and bikes_available > 3 and idle_time like '2018/07/29%'",conn)
HTML(idleTime_Station2.to_html(index=False))

idle_time,bikes_available


## Question-2 : In case two stations are nearby, it might be possible to shut one down. Find the distance between consecutive stations (between Stations 1 and 2, Stations 2 and 3, and so on) Using Haversine formula 

## Answer for this question alone please refer the .sql file shared along withe assignment. since sqllite3 does not support trignometric function had to go with mysql workbench

## Following is what was approach for this problem,
### Created a table using cross join and then applied the formula to check minimum distance

### below are quries(you will find the same in .sql file)
create table cross_j (
SELECT a.station_name station_a, 
b.station_name station_b, a.latitude station_a_lat, b.latitude station_b_lat,a.longitude station_a_long,b.longitude station_b_long 
FROM station AS a CROSS JOIN station AS b where a.station_name!=b.station_name
);

SELECT station_a,station_b,2 * 3961 * asin(sqrt(power((sin(radians((station_b_lat - station_a_lat) / 2))), 2) + cos(radians(station_a_lat)) * cos(radians(station_b_lat)) * power((sin(radians((station_b_long - station_a_long) / 2))), 2))) as distance
from cross_j order by distance;

##  Question 4 : three stations that can be shut

#### From the above results of most closest and Question-1 top 10 least popular stations
##### we see that
##### 'Redwood City Public Library' and'Franklin at Maple' are very close as well as not so popular 
##### 'Franklin at Maple' and 'San Mateo County Center' are very close as well as not so popular
##### 'Redwood City Public Library'and 'San Mateo County Center'are very close as well as not so popular
##### we can recommend Stations to be shutdown are,
1. Redwood City Public Library
2. Franklin at Maple
3. San Mateo County Center



# Task 3: Optimizing Operations

## Question-1: Calculate the average number of bikes and docks available for Station 2 and Station 3 (Hint: Use the Status table.)

In [143]:
avg_num = pd.read_sql_query("SELECT station_id ,round(avg(bikes_available)) as avg_bikes_available, round(avg(docks_available)) as avg_docks_available FROM Status where station_id in (2,3) group by station_id",conn)
HTML(avg_num.to_html(index=False))

station_id,avg_bikes_available,avg_docks_available
2,13.0,14.0
3,8.0,7.0


## Question 2 & 3

### Please refer Tableau file shared

## Question 4: Use the findings above to provide insights on how to optimize operations

### Answer: 
### From the above finding we could see which are the most popular areas and also through tableau map for customer & subscriber, we could see how popularity varies. To optimize and more profits we could increase the avg available bikes at most popular areas one such example is id = 70 (San Francisco Caltrain (Townsend at 4th)) in station table, this station is one of the most popular amoung subscribers, at this station we can increase avg available bikes

# Task 4: Couple Bikes? (Bonus)

### Zulip has decided to start a new product line called Couple Bikes. This will enable two persons to travel from one station to another at the same time. What are some of the factors that you will have to consider while validating the idea of couple bikes?

### factors to consider for validating the possibilty of couple bike are,
#### 1. If Start Station Name and End Station Name are same 
#### 2. Target the most popular areas as these areas will mostly be from where working professionals, students etc will travel.
#### 3. Cost when 2 person travelling in single bike must be less than the cost that 2 individual bikes would have cost
#### 4. Least Most popular areas where we must avoide couple bike implementation

## Lets discussion all 3 Factors one by one.....................

### Factor -1 : If Start Station Name and End Station Name are same

#### Traget above areas which had same start and end location these area had total 9,68,09,91,556 trips out of 3,08,43,70,70,970 trips in 3 years that is around 3 % of trips with same start and end location

In [137]:
sameLocation = pd.read_sql_query("SELECT start_station_name FROM Trip where start_station_name = end_station_name group by start_station_name",conn)
HTML(sameLocation.to_html(index=False))

start_station_name
2nd at Folsom
2nd at South Park
2nd at Townsend
5th at Howard
Adobe on Almaden
Arena Green / SAP Center
Beale at Market
Broadway St at Battery St
Broadway at Main
California Ave Caltrain Station


In [141]:
# All trips with same start and end location
AvgDuration_Location = pd.read_sql_query("SELECT sum(id) as total_trips FROM Trip where start_station_name = end_station_name ",conn)
HTML(AvgDuration_Location.to_html(index=False))

total_trips
9680991556


### Factor-2 : Target the most popular area as these area will mostly be from where working professionals, students etc will travel

#### Traget Areas such as,
#### 1. College/University from nearest metro stations
#### These are the areas which will have same start and end between college/university to metro vice versa.
#### 2. IT Tech parks / Hospitals / Major Bank Branches to nearest Metro Stations
#### These area are also those can be busy 24 hrs as they function 24hrs due to there work culture and since crowed travelling to & fro from metro stations to these areas is more frequent, its one of the best location to tackle
#### 3. Lastly Grocery/hypermart/Super market such as walmart to the nearest hosuing society/flats/apartment  residential areas
#### These are very good location which will target all group of people, since it involve day to day basic shopping

### Factor - 3 :Cost when 2 person travelling in single bike must be less than the cost that 2 individual bikes would have cos

#### Cost for transportation is one of the main factors that every individual co least mpares while opting the best options available in public transport
#### At the very least, an individual cost while in couple travel must be less than 25%. Beacuse this % shall give individual 25 % and Bike company will get 50%.
#### Lets say P1 and P2 are two individual who are travelling same destination, in normal situation it will cost them X amount each respectively i.e. 2X in Total and lets also assume travel cost of bike for bike comapny also as 2X.
#### Now if travelled in couple bike with 25% discount P1 and P2 Total cost would be 1.5X (0.75X each) and lets say travel cost of bike for bike comapny as X (assuming same amount from above for single bike), then we clearly see that profit for bike comapny also is 50%.

### Note: Factor 2 & 3 cannot be completely accurated/validated since these are just formualted on based market needs that we all see day to day also the above data's does not contain any such column through which we can check Cost or specific locations related to metro stations/universities/Markets

## Factor-4: Least most Popular area
### We must avoide those regions/stations which are least used for example below are the 10 most least popular areas based on amount of trips that take place

In [144]:
Ten_leastpopular = pd.read_sql_query("SELECT start_station_name ,Count(id) as total_trips from Trip group by start_station_name order by total_trips limit 10 ",conn)
HTML(Ten_leastpopular.to_html(index=False))

start_station_name,total_trips
San Jose Government Center,23
Broadway at Main,67
Redwood City Public Library,213
Franklin at Maple,224
San Mateo County Center,287
Redwood City Medical Center,311
Mezes Park,341
Stanford in Redwood City,436
Park at Olive,750
Santa Clara County Civic Center,840


## Conclusion:
### Two main Params must be considered while deciding couple bikes idea,
### 1st Location --- Which are the stations best fit and which are not based on the existing day to day trips data
### 2nd Cost/Price point for customers such that we can get win-win situation